In [1]:
# imports
%matplotlib inline


import seaborn as sns
import matplotlib.pyplot as plt
from pyrt.data.data_trots import *
from pyrt.optimization.vmat import *
from pyrt.tools import *

In [ ]:
def print_structure_info(data):
    for s in data.structures:
        print s.name,s.rx, s.num_vox, s.num_beamlets, s.is_target, s.Dij.shape
        print '-'*20

In [2]:
# Josh Inputs
# Find work directory
working_directory = 'outputs_josh/'
path1 = '/Users/jtmargo/Desktop/Josh/Clemson University/Research/Treatment Plan/TROTS Data/Prostate_VMAT/'
path2 = '/Users/jtmargo/Desktop/Josh/Clemson University/Research/Treatment Plan/TROTS Data/Head-and-Neck/'


#Set cwd
cwd = path1

# Load patient information



back_proj_list_of_dicts = [
   {'PTV':30.,'Bladder':15.,'default':0., 'threshold':0.},
    {'PTV':30.,'Rectum':15.,'default':0., 'threshold':0.},
#     {'PTV':5,'default':1, 'threshold':0.},
    {'PTV':100.,'default':1., 'threshold':0.},
    {'PTV':1.,'default':0., 'threshold':0.}
]

vmat_model_params = {
    'target_weights':{'PTV':1000., 'default':500.},
    'oar_weights':{'Rectum':10., 'Bladder':10., 'default':1.},
    'max_intensity':1000.,
    'min_intensity': 1.,
    'aper_limit': 1.,
    'max_distance_per_cp': 10000,
    'cp_redundancy': 1,
    'back_projection_dicts':back_proj_list_of_dicts
}


input_dict = {
    'cwd': cwd,
    'figure_directory':working_directory,
    'aper_types_list': ['back_proj'],
    'filename': None,
#     'Rx': {'PTV 0-46 Gy': 47.15},
    'Rx': {'PTV': 79.56, 'PTV Vesicles': 72.2},
    'model_params':vmat_model_params
}


In [4]:
input_dict['filename'] = 'Prostate_VMAT_101.mat'
input_dict['case_directory'] = input_dict['filename'][0:-4]+'/'
model = vmat_mip(input_dict)

---------------------
|  Reading in data  |
---------------------
Building Structures
importing PTV Dij as dense matrix, converting to sparse...
importing PTV Shell 50 mm Dij as dense matrix, converting to sparse...
importing External Ring 20 mm Dij as sparse matrix
importing PTV Shell 5 mm Dij as dense matrix, converting to sparse...
importing PTV Shell 15 mm Dij as dense matrix, converting to sparse...
importing PTV Shell 25 mm Dij as dense matrix, converting to sparse...
importing Rectum Dij as dense matrix, converting to sparse...
importing Anus Dij as dense matrix, converting to sparse...
importing Hip (L) Dij as dense matrix, converting to sparse...
importing Hip (R) Dij as dense matrix, converting to sparse...
importing Bladder Dij as dense matrix, converting to sparse...
importing Patient Dij as dense matrix, converting to sparse...
Building CP
----------------------------
|  Data reading completed  |
----------------------------
---------------------------
|  Building Gurobi M

In [5]:
model.optimize(run_tag=input_dict['filename'][0:-4])

Optimize a model with 237138 rows, 154806 columns and 1645647 nonzeros
Variable types: 154714 continuous, 92 integer (92 binary)
Coefficient statistics:
  Matrix range     [8e-09, 1e+03]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 3.9064e+08
Presolve removed 222377 rows and 144955 columns
Presolve time: 3.05s
Presolved: 14761 rows, 9851 columns, 137212 nonzeros
Variable types: 9828 continuous, 23 integer (23 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    2278    1.0710531e+07   1.154189e+04   0.000000e+00      5s
    7826    1.5885117e+07   5.422457e+02   0.000000e+00     10s
    9939    1.6176033e+07   0.000000e+00   0.000000e+00     11s

Root relaxation: objective 1.617603e+07, 9939 iterations, 8.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [ ]:
print data.f['patient/Beams'].keys()

In [ ]:
f = data.f
b = f['patient/Beams/BeamConfig']
field = np.asarray(f[b['Field'][0][0]])
print field[20][40]

In [ ]:
# Draw a heatmap with the numeric values in each cell
for i in range(b['Field'].size):
#     print np.asarray(f[b['Field'][i][0]]).shape
    plt.figure()
    sns.heatmap(np.asarray(f[b['Field'][i][0]])[15:24,25:50], annot=True, linewidths=.5)

In [ ]:
np.asarray(f[b['Field'][1][0]]).shape[0]

In [ ]:
field = np.asarray(f[b['Field'][1][0]])
print field
print np.where(field==1)
print int(np.where(field==1)[0])
print field[17][37]

print np.asarray(data.f['patient/Beams/ElementIndex'][0][0])

In [ ]:
#Find min and max row positions of field for all control points
min_row = np.asarray(f[b['Field'][0][0]]).shape[1]
max_row = 0
for cp in range(data.num_control_points):
    field = np.asarray(f[b['Field'][cp][0]])
    min_cp = int(np.where(field==1)[0])
    max_cp = int(np.where(field==np.asarray(data.f['patient/Beams/ElementIndex'][0][cp]))[0])
    if min_cp < min_row:
        min_row = min_cp
    if max_cp > max_row:
        max_row = max_cp
        
    #return min_row, max_row+1
max_row += 1      
print 'Min row = {}'.format(min_row)
print 'Max row = {}'.format(max_row)

In [ ]:
#Find leaf positions for each control point field
cp = 0
field = np.asarray(f[b['Field'][cp][0]])
left_leaf = [int(np.where(field==1)[0])]
left_leaf_index = [1]
print int(np.where(field==1)[1])

print np.where(field[16][:]>0)[0][0]

# print left_leaf
# print left_leaf_index

# print field[16][:]
# print np.argmax(field[16][:])

# print int(np.where(field==1)[1])
# print field[16][37]
# print int(np.where(field==np.argmax(field[16][:]))[1])
# print np.argmax(field[16][:])
# print field[16][43]

# print int(np.where(field==np.argmax(field[16][:]))[1]) - int(np.where(field==1)[0])

num_beamlets_in_row = [int(np.argmax(field[16][:])) - int(np.where(field==1)[1])+1]
print num_beamlets_in_row 

# for row in range(min,max):
#     left_leaf.append(int(np.where(field==1)[1]))
#     left_leaf_index.append(int(np.where(field==1)[1]))

In [ ]:
cp = 0
field = np.asarray(f[b['Field'][cp][0]])
left_leaf = []
left_leaf_index = []
num_beamlets_in_row = []
row_array = range(min_row,max_row)
# row = 16
for row in row_array:
    left_leaf.append(int(np.where(field[row][:]>0)[0][0]))
    left_leaf_index.append(field[row][np.where(field[row][:]>0)[0][0]])
    num_beamlets_in_row.append(int(np.argmax(field[row][:])) - int(np.where(field[row][:]>0)[0][0])+1)
    
    
    
print left_leaf
print left_leaf_index
print np.array(num_beamlets_in_row).sum()

for r in range(len(left_leaf)):
    print field[row_array[r]][left_leaf[r]:left_leaf[r]+num_beamlets_in_row[r]]

In [ ]:
cp = 0
field = np.asarray(data.f[b['Field'][cp][0]])
num_beamlets = np.asarray(data.f['patient/Beams/ElementIndex'][0][cp])
test = control_point(cp, field, 16, 23, 0, num_beamlets)